# 01 - Exploratory Data Analysis

This notebook performs initial exploration of MIMIC-IV data for biomarker discovery.

## Objectives
1. Load and inspect MIMIC-IV tables
2. Explore CBC lab values distribution
3. Analyze disease prevalence
4. Identify data quality issues

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from pathlib import Path

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

In [ ]:
# Set paths
DATA_DIR = Path('../data/raw')
CONFIG_DIR = Path('../configs')

print(f"Data directory: {DATA_DIR}")
print(f"Data exists: {DATA_DIR.exists()}")

# List available data files
if DATA_DIR.exists():
    print("\nAvailable files:")
    for f in DATA_DIR.glob("*.parquet"):
        print(f"  - {f.name}")

## 1. Load Configuration Files

In [ ]:
# Load CBC features config
with open(CONFIG_DIR / 'cbc_features.yaml', 'r') as f:
    cbc_config = yaml.safe_load(f)

# Load diseases config
with open(CONFIG_DIR / 'diseases.yaml', 'r') as f:
    diseases_config = yaml.safe_load(f)

print("CBC Features:")
for feature in cbc_config['cbc_features'].keys():
    print(f"  - {feature}")

## 2. Load MIMIC-IV Tables

In [ ]:
# Load patients
patients = pd.read_parquet(DATA_DIR / 'patients.parquet')
print(f"Patients: {len(patients):,} rows")
patients.head()

In [ ]:
# Load admissions
admissions = pd.read_parquet(DATA_DIR / 'admissions.parquet')
print(f"Admissions: {len(admissions):,} rows")
admissions.head()

In [ ]:
# Load diagnoses
diagnoses = pd.read_parquet(DATA_DIR / 'diagnoses_icd.parquet')
print(f"Diagnoses: {len(diagnoses):,} rows")
diagnoses.head()

In [ ]:
# Load lab items dictionary
d_labitems = pd.read_parquet(DATA_DIR / 'd_labitems.parquet')
print(f"Lab items: {len(d_labitems):,} rows")
d_labitems.head(10)

In [ ]:
# Load lab events (may take a moment - large file)
labevents = pd.read_parquet(DATA_DIR / 'labevents.parquet')
print(f"Lab events: {len(labevents):,} rows")
labevents.head()

## 3. Explore CBC Lab Values

In [ ]:
# Get CBC itemids from config
cbc_itemids = []
itemid_to_name = {}

for feature_name, feature_info in cbc_config['cbc_features'].items():
    for itemid in feature_info['itemids']:
        cbc_itemids.append(itemid)
        itemid_to_name[itemid] = feature_name

print(f"CBC itemids: {cbc_itemids}")

In [ ]:
# Filter to CBC labs only
cbc_labs = labevents[labevents['itemid'].isin(cbc_itemids)].copy()
cbc_labs['feature_name'] = cbc_labs['itemid'].map(itemid_to_name)

print(f"CBC lab events: {len(cbc_labs):,} rows")
cbc_labs.head()

In [ ]:
# Summary statistics per CBC feature
cbc_summary = cbc_labs.groupby('feature_name')['valuenum'].describe()
cbc_summary

In [ ]:
# Plot distributions
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for idx, feature_name in enumerate(cbc_labs['feature_name'].unique()):
    if idx >= len(axes):
        break
    
    data = cbc_labs[cbc_labs['feature_name'] == feature_name]['valuenum']
    
    # Get reference range from config
    if feature_name in cbc_config['cbc_features']:
        ref_range = cbc_config['cbc_features'][feature_name].get('reference_range', {})
        low = ref_range.get('low')
        high = ref_range.get('high')
    else:
        low, high = None, None
    
    ax = axes[idx]
    ax.hist(data, bins=50, edgecolor='black', alpha=0.7)
    ax.set_title(feature_name)
    ax.set_xlabel('Value')
    ax.set_ylabel('Count')
    
    # Add reference range lines
    if low:
        ax.axvline(low, color='red', linestyle='--', label=f'Low: {low}')
    if high:
        ax.axvline(high, color='red', linestyle='--', label=f'High: {high}')
    
    ax.legend(fontsize=8)

# Hide empty subplots
for idx in range(len(cbc_labs['feature_name'].unique()), len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.savefig('../experiments/cbc_distributions.png', dpi=150)
plt.show()

## 4. Disease Prevalence Analysis

In [ ]:
# Count patients per disease
disease_counts = {}

for disease_name, disease_info in diseases_config['diseases'].items():
    icd_codes = disease_info.get('icd9_codes', []) + disease_info.get('icd10_codes', [])
    
    # Match patients with these ICD codes
    mask = diagnoses['icd_code'].str.startswith(tuple(icd_codes))
    patient_count = diagnoses[mask]['subject_id'].nunique()
    
    disease_counts[disease_name] = patient_count

disease_df = pd.DataFrame({
    'disease': disease_counts.keys(),
    'patient_count': disease_counts.values()
}).sort_values('patient_count', ascending=False)

disease_df

In [ ]:
# Plot disease prevalence
plt.figure(figsize=(12, 6))
plt.barh(disease_df['disease'], disease_df['patient_count'])
plt.xlabel('Number of Patients')
plt.ylabel('Disease')
plt.title('Disease Prevalence in MIMIC-IV')
plt.tight_layout()
plt.savefig('../experiments/disease_prevalence.png', dpi=150)
plt.show()

## 5. Data Quality Check

In [ ]:
# Check for missing values
print("Missing values in CBC labs:")
print(cbc_labs.isnull().sum())

In [ ]:
# Check for outliers
print("\nPotential outliers (beyond 3 std):")
for feature in cbc_labs['feature_name'].unique():
    data = cbc_labs[cbc_labs['feature_name'] == feature]['valuenum']
    mean, std = data.mean(), data.std()
    outliers = ((data < mean - 3*std) | (data > mean + 3*std)).sum()
    pct = 100 * outliers / len(data)
    print(f"  {feature}: {outliers:,} ({pct:.2f}%)")

## Next Steps

1. Implement data preprocessing pipeline
2. Create patient-level feature aggregation
3. Generate biomarker labels using threshold generators
4. Build baseline prediction models